In [1]:
# import GUI
from tkinter import *
from tkinter import filedialog
import tkinter.messagebox
from PIL import Image, ImageTk

# import data stuff
import pandas as pd
from pandas import DataFrame, read_csv

# import os stuff for cwd, mkdir, make output directory
import os
outputpath = 'EOLoutput'
if not os.path.exists(os.getcwd()+'/'+outputpath):
    os.mkdir(outputpath)
    
# import file name parsing
import ntpath
ntpath.basename("a/b/c") # ntpath.split(filename1) returns list of [path, filename]


# functions for input/output file selection
def browsefiles_in():
    global filename1p # previous filename1 path
    global filename1 # global in order to access outside of this function
    global uround
    global colour1
    global colour2
    if (ntpath.split(filename2)[0] == ""):
        showpath = "./"
    else:
        showpath = ntpath.split(filename2)[0]

    filename1 = filedialog.askopenfilename(initialdir = showpath, title = "select GP12 file", filetypes=(("excel files", "*.xls"), ("all files","*.*")))
    if (filename1 == filename1p):
        tkinter.messagebox.showinfo('Warning', 'New file selected is the same as previously selected file')
    filename1p = filename1
    if (uround % 2 == 0):
        label_file_explorer.configure(text=filename1, fg="blue", bg=colour1)
    else:
        label_file_explorer.configure(text=filename1, fg="blue", bg=colour2)
        
    
def browsefiles_out():
    global filename1
    global filename2p
    global filename2
    global filename2o # filename2, tail part of path, only filename
    global uround
    global colour1
    global colour2
    # show the most recent folder
    if (ntpath.split(filename1)[0] == ""):
        showpath = "./"
    else:
        showpath = ntpath.split(filename1)[0]
            
    filename2 = filedialog.askopenfilename(initialdir = showpath, title = "select EOL file", filetypes=(("csv files", "*.csv"), ("all files","*.*")))
    if (filename2 == filename2p):
        tkinter.messagebox.showinfo('Warning', 'New file selected is the same as previously selected file')
    filename2p = filename2
    if (uround % 2 == 0):
        label_file_explorer2.configure(text=filename2, fg="blue", bg=colour1)
    else:
        label_file_explorer2.configure(text=filename2, fg="blue", bg=colour2)
    filename2o = ntpath.split(filename2)[1]
    
# function to fetch excel sheet data -> write into csv file
def updateinfo():
    global filename1
    global filename2 # required to modify output file path 
    global filename2o # only filename, will be modified for saving
    global uround
    
    if (filename1 == "" or filename2 == ""):
        label_update.configure(text="FILENAME NOT SPECIFIED", fg="white", bg="red")
        return False

    # input at G4 G6 G7 at sheet MFG LIMITS , check discrepancy at sheet PARTNUMBER
    df_sheet_multi = pd.read_excel(filename1, sheet_name=['MFG LIMITS'], usecols='G',userows=0)
    df_sheet_multi0 = pd.read_excel(filename1, sheet_name=['PARTNUMBER'], usecols='A',userows=1)
    
    # selecting row removes dictionary key, returns dataframe
    E1 = df_sheet_multi.get("MFG LIMITS")
    E2 = df_sheet_multi0.get("PARTNUMBER")

    # convert dataframe into series by selecting rows
    D1 = (E1.loc[2])[0] # HP_APP, HP_BOOT
    D2 = (E1.loc[4])[0] # DSP_APP
    D3 = (E1.loc[5])[0] # DSP_BOOT
    D40 = (E2.loc[0])[0] # PARTNUMBER from sheet A
    D4 = (E1.loc[0]).index[0] # PARTNUMBER from sheet B
    D41 = (filename1[-14:])[:-4] # partnumber from INPUT filename GP12
    D5 = (filename2[-14:])[:-4] # partnumber from OUTPUT filename EOL

    ### Warning messages, does nothing except showing pop-up box
    ###  if Sheet'PARTNUMBER', Sheet'MFG LIMITS', or Filename'PARTNUMBER'.xls does not match
    if (D40 != D4 or D4 != D41 or D40 != D41):
        tkinter.messagebox.showinfo('Warning','GP12 partnumber discrepancy, update halted')
        return False
        
    ###  if any of GP12 part number does not match EOL Filename'PARTNUMBER'.csv    
    if (D4 != D5 or D40 != D5 or D41 != D5):
        tkinter.messagebox.showinfo('Warning','EOL filename updated')
        ######## keep old file or just create updated file in addition? ########
        ### output file to new folder
    
    # Data fetched from input file, GP12
    D11 = str(int(D1[:2],16)) + '.' + str(int(D1[2:4],16)) + '.0' # HP_APP
    D22 = str(int(D1[4:6],16)) + '.' + str(int(D1[6:8],16)) + '.0' # HP_BOOT
    D33 = str(int(D2[:2],16)) + '.' + str(int(D2[2:4],16)) + '.' + str(int(D2[4:6],16)) + '.' + str(int(D2[6:8],16)) + '.' + str(int(D2[8:10],16)) # DSP_APP
    D44 = str(int(D3[:2],16)) + '.' + str(int(D3[2:4],16)) + '.' + str(int(D3[4:6],16)) + '.' + str(int(D3[6:8],16)) + '.' + str(int(D3[8:10],16)) # DSP_BOOT
    D55 = str(D4[:7] + ' ' + D4[7:9] + ' ' + D4[9:]) # Formatted partnumber from GP12.'MFG LIMITS' -> NB[F1]
    
    # First line has part number and revision number, save for writing output file
    df_csv_out_first = pd.read_csv(filename2, nrows=1, header=None)
    
    # loading the rest of EOL output file, label of columns on row 2, do not touch NA entries
    df_csv_out = pd.read_csv(filename2, header=1, na_filter=False)
    
    # Update values
    (df_csv_out.loc[df_csv_out['Requirement'] == 'HP_APP', 'Min']) = D11
    (df_csv_out.loc[df_csv_out['Requirement'] == 'HP_BOOT', 'Min']) = D22
    (df_csv_out.loc[df_csv_out['Requirement'] == 'DSP_APP', 'Min']) = D33
    (df_csv_out.loc[df_csv_out['Requirement'] == 'DSP_BOOT', 'Min']) = D44
    df_csv_out_first.iloc[0,5] = D55 # D55 value actually written on output file now
    
    
    ### NEW file for output is ALWAYS created in output folder
    ### NEW filename for EOL output, based on D4 = GP12.'MFG LIMITS'[G1]
    filename2o = filename2o[:-14] # remove .csv extension & 10 digit part number
    filename2o = filename2o + D4 + '.csv' # path + partnumber + extension
    
    # write to new CSV file, create file if does not exist, write first line, append the rest
    df_csv_out_first.to_csv(os.getcwd() + '/' + outputpath +'/'+ filename2o, index=False, header=False, mode='w')
    df_csv_out.to_csv(os.getcwd() + '/' + outputpath +'/'+ filename2o, index=False, header=True, mode='a')
    uround = uround + 1
    label_update.configure(text="update successful! [# files processed = " + str(uround) + "]", fg="white", bg="green")

def ExitApplication() :
    msgbox = tkinter.messagebox.askquestion ('Exit Application', 'Are you sure?', icon='warning')
    if msgbox == 'yes':
        window.destroy()
    else:
        return False
    
# update round, to colour background
uround = 0 # keep track of how many files processed
colour1 = "palegreen" # successful file pick, first iteration
colour2 = "greenyellow" # successful file pick, second iteration
filename1 = ""
filename2 = ""
filename1p = ""
filename2p = ""

# GUI body
window = Tk()
window.title('EOL File Generator')
window.geometry("932x220")
window.config(background="white")

# GUI labels
label_file_explorer = Label(window, text="GP12 path", width = 110, height = 3, fg = "blue", bg = "white")
label_file_explorer2 = Label(window, text="EOL path", width = 110, height = 3, fg = "blue", bg = "white")
label_update = Label(window, text="", width=110, height=3, fg="blue", bg = "white")
label_file1 = Label(window, text="GP12", width = 10, height = 3, fg = "black", bg = "white")
label_file2 = Label(window, text="EOL", width = 10, height = 3, fg = "black", bg = "white")
label_status = Label(window, text="Status", width=10, height=3, fg="black", bg = "white")
label_info = Label(window, text="Version", width=10, height=3, fg="black", bg = "white")

# GUI buttons
button_explore = Button(window, text="1. select GP12", command=browsefiles_in)
button_explore2 = Button(window, text="2. select EOL", command=browsefiles_out)
button_update = Button(window,text="3. update",command=updateinfo)
button_exit = Button(window,text="4. exit",command=ExitApplication)
button_explore.config(width = 10, height = 3, bg="CadetBlue2")
button_explore2.config(width = 10, height = 3, bg="CadetBlue2")
button_update.config(width = 10, height = 3, bg="hot pink")
button_exit.config(width = 10, height = 3, bg="DarkOrange4")

# GUI arrangement
label_file1.grid(column=1, row=1)
label_file2.grid(column=1, row=2)
label_status.grid(column=1,row=3)
label_info.grid(column=1,row=4)
label_file_explorer.grid(column=2, row=1)
label_file_explorer2.grid(column=2, row=2)
label_update.grid(column=2,row=3)
label_info.grid(column=2,row=4, sticky=W)
button_explore.grid(column=3, row=1)
button_explore2.grid(column=3, row=2)
button_update.grid(column=3, row=3)
button_exit.grid(column=3, row=4)

window.mainloop()